In [13]:
# Dependencies

# from config import (census_key, gkey)
from scipy.stats import linregress
import statsmodels.api as sm

import gmaps
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
import time

import requests
from pprint import pprint

In [17]:
URL = 'https://api.greatschools.org/schools/CA/1?key=71c0733d882cd6461374cc728193ed49'
api_key = '71c0733d882cd6461374cc728193ed49'

response = requests.get(URL)
response.content

b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><school><gsId>1</gsId><name>Alameda High School</name><type>public</type><gradeRange>9-12</gradeRange><enrollment>1853</enrollment><gsRating>9</gsRating><city>Alameda</city><state>CA</state><districtId>1</districtId><district>Alameda Unified School District</district><districtNCESId>0601770</districtNCESId><address>2201 Encinal Avenue, \nAlameda, CA  94501</address><phone>(510) 337-7022</phone><fax>(510) 521-4740</fax><website>http://aus.alamedausd.ca.schoolloop.com/</website><ncesId>060177000041</ncesId><lat>37.764294</lat><lon>-122.24807</lon><overviewLink>https://www.greatschools.org/california/alameda/1-Alameda-High-School/?utm_source=GSAPI&amp;utm_medium=referral</overviewLink><ratingsLink>https://www.greatschools.org/california/alameda/1-Alameda-High-School/?utm_source=GSAPI&amp;utm_medium=referral</ratingsLink><reviewsLink>https://www.greatschools.org/california/alameda/1-Alameda-High-School/?utm_source=GSAPI&amp;utm_mediu